In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from skimage import io
from PIL import Image, ImageOps, UnidentifiedImageError
import requests
from io import BytesIO

In [2]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import pylab

We don't want to use scans that have no Majority Label nor Correct Label as these still need to be looked at by
neurosurgeons to verify if they have hemorrhaging or not

In [3]:
epi = pd.read_csv("Results_Epidural Hemorrhage Detection_2020-11-16_21.31.26.148.csv", error_bad_lines=False)
clean_epi = epi[(epi['Majority Label'] != '[]') | (epi['Correct Label'].notna()) & (epi['Correct Label'] != '[[], []]') ]

C:\Users\sahsa\AppData\Local\Temp\ipykernel_31712\3674797304.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  epi = pd.read_csv("Results_Epidural Hemorrhage Detection_2020-11-16_21.31.26.148.csv", error_bad_lines=False)


In [4]:
clean_epi.head()

,Case ID,Origin,URL,Labeling State,All Labels,Total Reads,Total Qualified Reads,Majority Label,Number of ROIs,Correct Label,Difficulty,Agreement,Comments,Internal Notes
0,1196514,ID_004c4b319.jpg,https://go.centaurlabs.com/problem/1196514,Labeled,"['[{""x"": 0.40818718, ""y"": 0.8623607}, {""x"": 0....",19.0,5.0,"[{""x"": 0.537109375, ""y"": 0.89453125}, {""x"": 0....",1.0,"[[{'x': 0.537109375, 'y': 0.89453125}, {'x': 0...",0.00,0.968851,NaN,NaN
1,1196682,ID_0237b835f.jpg,https://go.centaurlabs.com/problem/1196682,Ready,"['[{""x"": 0.62959146, ""y"": 0.720186}, {""x"": 0.6...",51.0,28.0,"[{""x"": 0.654296875, ""y"": 0.724609375}, {""x"": 0...",1.0,NaN,NaN,0.063519,NaN,NaN
2,1196477,ID_02e2d273d.jpg,https://go.centaurlabs.com/problem/1196477,Labeled,"['[{""x"": 0.35205495, ""y"": 0.34680602}, {""x"": 0...",22.0,6.0,"[{""x"": 0.46484375, ""y"": 0.357421875}, {""x"": 0....",1.0,"[[{'x': 0.46484375, 'y': 0.357421875}, {'x': 0...",0.06,0.710300,NaN,NaN
3,1196612,ID_02f37ce84.jpg,https://go.centaurlabs.com/problem/1196612,Ready,"['[{""x"": 0.3680166, ""y"": 0.48387372}, {""x"": 0....",51.0,29.0,"[{""x"": 0.431640625, ""y"": 0.607421875}, {""x"": 0...",2.0,NaN,NaN,0.264134,NaN,NaN
4,1196612,ID_02f37ce84.jpg,https://go.centaurlabs.com/problem/1196612,Ready,"['[{""x"": 0.3680166, ""y"": 0.48387372}, {""x"": 0....",51.0,29.0,"[{""x"": 0.353515625, ""y"": 0.59765625}, {""x"": 0....",2.0,NaN,NaN,0.264134,NaN,NaN


In [5]:
intra = pd.read_csv("Results_Intraparenchymal Hemorrhage Detection_2020-11-16_21.39.31.268.csv",error_bad_lines=False)
clean_intra = intra[(intra['Majority Label'] != '[]') | (intra['Correct Label'].notna()) & (intra['Correct Label'] != '[[]]') ]
#not sure what to do with scans that have "Gold Standard" (as their labeling state) since some of these have nans for both
#Majority Label and Correct Label
intra.shape
#original excel file has 1373 rows, not exactly sure why pd only reads 1289 even after deleting
#the "error_bad_lines=False" command

C:\Users\sahsa\AppData\Local\Temp\ipykernel_31712\3798969853.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  intra = pd.read_csv("Results_Intraparenchymal Hemorrhage Detection_2020-11-16_21.39.31.268.csv",error_bad_lines=False)


(1289, 14)

In [6]:
clean_intra.head()

,Case ID,Origin,URL,Labeling State,All Labels,Total Reads,Total Qualified Reads,Majority Label,Number of ROIs,Correct Label,Difficulty,Agreement,Comments,Internal Notes
0,1675660,ID_0002081b6.jpg,https://go.centaurlabs.com/problem/1675660,In Progress,"['[{""x"": 0.31658885, ""y"": 0.48357323}, {""x"": 0...",22.0,9.0,"[{""x"": 0.3046875, ""y"": 0.484375}, {""x"": 0.3046...",1.0,NaN,NaN,0.224542,NaN,NaN
1,1675942,ID_0048e60a4.jpg,https://go.centaurlabs.com/problem/1675942,In Progress,"['[{""x"": 0.54811704, ""y"": 0.5961946}, {""x"": 0....",22.0,7.0,"[{""x"": 0.5859375, ""y"": 0.640625}, {""x"": 0.5859...",1.0,NaN,NaN,0.772365,NaN,NaN
2,1197689,ID_0057d2736.jpg,https://go.centaurlabs.com/problem/1197689,Labeled,"['[{""x"": 0.3040109, ""y"": 0.52168596}, {""x"": 0....",46.0,9.0,"[{""x"": 0.3046875, ""y"": 0.521484375}, {""x"": 0.3...",1.0,"[[{'x': 0.3046875, 'y': 0.521484375}, {'x': 0....",0.02,0.738368,NaN,NaN
3,1197754,ID_007cac3b7.jpg,https://go.centaurlabs.com/problem/1197754,Labeled,"['[{""x"": 0.41792268, ""y"": 0.56897116}, {""x"": 0...",51.0,10.0,"[{""x"": 0.41015625, ""y"": 0.615234375}, {""x"": 0....",1.0,"[[{'x': 0.4140625, 'y': 0.6171875}, {'x': 0.41...",0.03,0.618086,NaN,NaN
4,1675775,ID_00b46c6bc.jpg,https://go.centaurlabs.com/problem/1675775,In Progress,"['[{""x"": 0.82215744, ""y"": 0.20116618}, {""x"": 0...",22.0,6.0,"[{""x"": 0.365234375, ""y"": 0.693359375}, {""x"": 0...",1.0,NaN,NaN,0.006388,NaN,NaN


In [7]:
multiple = pd.read_csv("Results_Multiple Hemorrhage Detection_2020-11-16_21.36.24.018.csv",error_bad_lines=False )
#again, excel file has more rows but pd reading less
clean_multiple = multiple[(multiple['Majority Label'] != '[]') | (multiple['Correct Label'].notna()) & (multiple['Correct Label'] != '[[]]') ]
multiple.shape

C:\Users\sahsa\AppData\Local\Temp\ipykernel_31712\2722983080.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  multiple = pd.read_csv("Results_Multiple Hemorrhage Detection_2020-11-16_21.36.24.018.csv",error_bad_lines=False )


(1054, 14)

In [8]:
clean_multiple.head()

,Case ID,Origin,URL,Labeling State,All Labels,Total Reads,Total Qualified Reads,Majority Label,Number of ROIs,Correct Label,Difficulty,Agreement,Comments,Internal Notes
0,1198534,ID_01af922c3.jpg,https://go.centaurlabs.com/problem/1198534,In Progress,"['[{""x"": 0.372072, ""y"": 0.6399058}, {""x"": 0.36...",23.0,6.0,"[{""x"": 0.396484375, ""y"": 0.73046875}, {""x"": 0....",3.0,NaN,NaN,0.790347,NaN,NaN
1,1198534,ID_01af922c3.jpg,https://go.centaurlabs.com/problem/1198534,In Progress,"['[{""x"": 0.372072, ""y"": 0.6399058}, {""x"": 0.36...",23.0,6.0,"[{""x"": 0.619140625, ""y"": 0.765625}, {""x"": 0.61...",3.0,NaN,NaN,0.790347,NaN,NaN
2,1198534,ID_01af922c3.jpg,https://go.centaurlabs.com/problem/1198534,In Progress,"['[{""x"": 0.372072, ""y"": 0.6399058}, {""x"": 0.36...",23.0,6.0,"[{""x"": 0.46875, ""y"": 0.607421875}, {""x"": 0.468...",3.0,NaN,NaN,0.790347,NaN,NaN
3,1198791,ID_01c457a64.jpg,https://go.centaurlabs.com/problem/1198791,Labeled,"['[{""x"": 0.40734115, ""y"": 0.5586468}, {""x"": 0....",34.0,6.0,"[{""x"": 0.4375, ""y"": 0.7109375}, {""x"": 0.4375, ...",3.0,"[[{'x': 0.548828125, 'y': 0.6796875}, {'x': 0....",0.48,0.124994,NaN,NaN
4,1198791,ID_01c457a64.jpg,https://go.centaurlabs.com/problem/1198791,Labeled,"['[{""x"": 0.40734115, ""y"": 0.5586468}, {""x"": 0....",34.0,6.0,"[{""x"": 0.546875, ""y"": 0.671875}, {""x"": 0.54687...",3.0,"[[{'x': 0.548828125, 'y': 0.6796875}, {'x': 0....",0.48,0.124994,NaN,NaN


In [9]:
subarachnoid = pd.read_csv("Results_Subarachnoid Hemorrhage Detection_2020-11-16_21.36.18.668.csv", error_bad_lines=False)
clean_subarachnoid = subarachnoid[(subarachnoid['Majority Label'] != '[]') | (subarachnoid['Correct Label'].notna()) & (subarachnoid['Correct Label'] != '[[]]') ]
subarachnoid.shape
#original excel file has 1054 rows, not exactly sure why pd only reads 1031 even after deleting
#the "error_bad_lines=False" command

C:\Users\sahsa\AppData\Local\Temp\ipykernel_31712\1871949402.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  subarachnoid = pd.read_csv("Results_Subarachnoid Hemorrhage Detection_2020-11-16_21.36.18.668.csv", error_bad_lines=False)


(1031, 14)

In [10]:
clean_subarachnoid.head()

,Case ID,Origin,URL,Labeling State,All Labels,Total Reads,Total Qualified Reads,Majority Label,Number of ROIs,Correct Label,Difficulty,Agreement,Comments,Internal Notes
0,1197112,ID_001b1b944.jpg,https://go.centaurlabs.com/problem/1197112,In Progress,"['[{""x"": 0.43056944, ""y"": 0.5936399}, {""x"": 0....",16.0,5.0,"[{""x"": 0.314453125, ""y"": 0.75}, {""x"": 0.314453...",3.0,NaN,NaN,0.252748,NaN,NaN
1,1197112,ID_001b1b944.jpg,https://go.centaurlabs.com/problem/1197112,In Progress,"['[{""x"": 0.43056944, ""y"": 0.5936399}, {""x"": 0....",16.0,5.0,"[{""x"": 0.400390625, ""y"": 0.796875}, {""x"": 0.40...",3.0,NaN,NaN,0.252748,NaN,NaN
2,1197112,ID_001b1b944.jpg,https://go.centaurlabs.com/problem/1197112,In Progress,"['[{""x"": 0.43056944, ""y"": 0.5936399}, {""x"": 0....",16.0,5.0,"[{""x"": 0.556640625, ""y"": 0.302734375}, {""x"": 0...",3.0,NaN,NaN,0.252748,NaN,NaN
3,1197367,ID_005f428d2.jpg,https://go.centaurlabs.com/problem/1197367,Labeled,"['[{""x"": 0.59604716, ""y"": 0.4975766}, {""x"": 0....",29.0,5.0,"[{""x"": 0.54296875, ""y"": 0.763671875}, {""x"": 0....",2.0,"[[{'x': 0.626953125, 'y': 0.30078125}, {'x': 0...",0.33,0.763632,NaN,NaN
4,1197367,ID_005f428d2.jpg,https://go.centaurlabs.com/problem/1197367,Labeled,"['[{""x"": 0.59604716, ""y"": 0.4975766}, {""x"": 0....",29.0,5.0,"[{""x"": 0.630859375, ""y"": 0.30078125}, {""x"": 0....",2.0,"[[{'x': 0.626953125, 'y': 0.30078125}, {'x': 0...",0.33,0.763632,NaN,NaN


In [11]:
subdural1 = pd.read_csv("Results_Subdural Hemorrhage Detection_2020-11-16_21.35.48.040.csv",error_bad_lines=False)
clean_subdural1 = subdural1[(subdural1['Majority Label'] != '[]') | (subdural1['Correct Label'].notna()) & (subdural1['Correct Label'] != '[[]]')]

C:\Users\sahsa\AppData\Local\Temp\ipykernel_31712\3142101893.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  subdural1 = pd.read_csv("Results_Subdural Hemorrhage Detection_2020-11-16_21.35.48.040.csv",error_bad_lines=False)


In [12]:
clean_subdural1.head()

,Case ID,Origin,URL,Labeling State,All Labels,Total Reads,Total Qualified Reads,Majority Label,Number of ROIs,Correct Label,Difficulty,Agreement,Comments,Internal Notes
0,1198320,ID_0061cb462.jpg,https://go.centaurlabs.com/problem/1198320,Labeled,"['[{""x"": 0.4823523, ""y"": 0.59177}, {""x"": 0.473...",16.0,5.0,"[{""x"": 0.45703125, ""y"": 0.78125}, {""x"": 0.4570...",2.0,"[[{'x': 0.40625, 'y': 0.330078125}, {'x': 0.40...",0.29,0.949092,NaN,NaN
1,1198320,ID_0061cb462.jpg,https://go.centaurlabs.com/problem/1198320,Labeled,"['[{""x"": 0.4823523, ""y"": 0.59177}, {""x"": 0.473...",16.0,5.0,"[{""x"": 0.404296875, ""y"": 0.333984375}, {""x"": 0...",2.0,"[[{'x': 0.40625, 'y': 0.330078125}, {'x': 0.40...",0.29,0.949092,NaN,NaN
2,1198004,ID_0086b5e67.jpg,https://go.centaurlabs.com/problem/1198004,In Progress,"['[{""x"": 0.5224404, ""y"": 0.8304443}, {""x"": 0.5...",15.0,6.0,"[{""x"": 0.521484375, ""y"": 0.83203125}, {""x"": 0....",2.0,NaN,NaN,0.157037,NaN,NaN
3,1198004,ID_0086b5e67.jpg,https://go.centaurlabs.com/problem/1198004,In Progress,"['[{""x"": 0.5224404, ""y"": 0.8304443}, {""x"": 0.5...",15.0,6.0,"[{""x"": 0.478515625, ""y"": 0.83984375}, {""x"": 0....",2.0,NaN,NaN,0.157037,NaN,NaN
4,1198353,ID_00eb6f7cc.jpg,https://go.centaurlabs.com/problem/1198353,Labeled,"['[{""x"": 0.48163298, ""y"": 0.71568763}, {""x"": 0...",15.0,5.0,"[{""x"": 0.447265625, ""y"": 0.73828125}, {""x"": 0....",1.0,"[[{'x': 0.447265625, 'y': 0.73828125}, {'x': 0...",0.00,0.306828,NaN,NaN


In [13]:
subdural2 = pd.read_csv("Results_Subdural Hemorrhage Detection_2020-11-16_21.37.19.745.csv",error_bad_lines=False)
clean_subdural2 = subdural2[(subdural2['Majority Label'] != '[]') | (subdural2['Correct Label'].notna()) & (subdural2['Correct Label'] != '[[]]')]

C:\Users\sahsa\AppData\Local\Temp\ipykernel_31712\909399964.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  subdural2 = pd.read_csv("Results_Subdural Hemorrhage Detection_2020-11-16_21.37.19.745.csv",error_bad_lines=False)


In [14]:
clean_subdural2.head()

,Case ID,Origin,URL,Labeling State,All Labels,Total Reads,Total Qualified Reads,Majority Label,Number of ROIs,Correct Label,Difficulty,Agreement,Comments,Internal Notes
0,1198320,ID_0061cb462.jpg,https://go.centaurlabs.com/problem/1198320,Labeled,"['[{""x"": 0.4823523, ""y"": 0.59177}, {""x"": 0.473...",16.0,5.0,"[{""x"": 0.45703125, ""y"": 0.78125}, {""x"": 0.4570...",2.0,"[[{'x': 0.40625, 'y': 0.330078125}, {'x': 0.40...",0.29,0.949092,NaN,NaN
1,1198320,ID_0061cb462.jpg,https://go.centaurlabs.com/problem/1198320,Labeled,"['[{""x"": 0.4823523, ""y"": 0.59177}, {""x"": 0.473...",16.0,5.0,"[{""x"": 0.404296875, ""y"": 0.333984375}, {""x"": 0...",2.0,"[[{'x': 0.40625, 'y': 0.330078125}, {'x': 0.40...",0.29,0.949092,NaN,NaN
2,1198004,ID_0086b5e67.jpg,https://go.centaurlabs.com/problem/1198004,In Progress,"['[{""x"": 0.5224404, ""y"": 0.8304443}, {""x"": 0.5...",15.0,6.0,"[{""x"": 0.521484375, ""y"": 0.83203125}, {""x"": 0....",2.0,NaN,NaN,0.157037,NaN,NaN
3,1198004,ID_0086b5e67.jpg,https://go.centaurlabs.com/problem/1198004,In Progress,"['[{""x"": 0.5224404, ""y"": 0.8304443}, {""x"": 0.5...",15.0,6.0,"[{""x"": 0.478515625, ""y"": 0.83984375}, {""x"": 0....",2.0,NaN,NaN,0.157037,NaN,NaN
4,1198353,ID_00eb6f7cc.jpg,https://go.centaurlabs.com/problem/1198353,Labeled,"['[{""x"": 0.48163298, ""y"": 0.71568763}, {""x"": 0...",15.0,5.0,"[{""x"": 0.447265625, ""y"": 0.73828125}, {""x"": 0....",1.0,"[[{'x': 0.447265625, 'y': 0.73828125}, {'x': 0...",0.00,0.306828,NaN,NaN


In [15]:
# adding label column to all the dataframes
# ignore the warnings

# mapping of dummy label to class
label_key = {0: "epi", 1: "intra", 2: "multi", 3: "subarachnoid", 4: "subdural1", 5: "subdural2"}

clean_epi["label"] = 0
clean_intra["label"] = 1
clean_multiple["label"] = 2
clean_subarachnoid["label"] = 3
clean_subdural1["label"] = 4
clean_subdural2["label"] = 5

C:\Users\sahsa\AppData\Local\Temp\ipykernel_31712\3286737939.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_epi["label"] = 0
C:\Users\sahsa\AppData\Local\Temp\ipykernel_31712\3286737939.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_intra["label"] = 1
C:\Users\sahsa\AppData\Local\Temp\ipykernel_31712\3286737939.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [16]:
# removing flagged images
with open("flagged.txt", "r") as f:
    lines = f.readlines()
flagged = []
for l in lines:
    flagged.append(l.strip())
clean_epi = clean_epi.drop(clean_epi[clean_epi["Origin"].isin(flagged)].index)
clean_intra = clean_intra.drop(clean_intra[clean_intra["Origin"].isin(flagged)].index)
clean_multiple = clean_multiple.drop(clean_multiple[clean_multiple["Origin"].isin(flagged)].index)
clean_subarachnoid = clean_subarachnoid.drop(clean_subarachnoid[clean_subarachnoid["Origin"].isin(flagged)].index)
clean_subdural2 = clean_subdural2.drop(clean_subdural2[clean_subdural2["Origin"].isin(flagged)].index)

In [17]:
#adding images to the cleaned epidural dataframe
root_url = 'https://raw.githubusercontent.com/Red-Nova/MATH4570-Project-Data/main/BrainCT/'
img_list = []
for class_name,df in ({'epidural':clean_epi}).items():

    for idx,fname in enumerate(df['Origin']):
        #for now, only process the brain window
        response = requests.get(url=f'{root_url}{class_name}/{class_name}/brain_window/{fname}')
        img = Image.open(BytesIO(response.content))
        img = ImageOps.grayscale(img)
        img = np.asarray(img)
        img = np.reshape(img, (512*512))
        img_list.append(img)

    img_df = pd.DataFrame({'image_matrix': img_list})
    clean_epi = pd.concat([df, img_df], axis=1)
    img_list = [] #reset the image list for the next dataframe

In [18]:
#adding images to the cleaned intraparenchymal dataframe
root_url = 'https://raw.githubusercontent.com/Red-Nova/MATH4570-Project-Data/main/BrainCT/'
img_list = []
for class_name,df in ({'intraparenchymal':clean_intra}).items():

    for idx,fname in enumerate(df['Origin']):
        #for now, only process the brain window
        response = requests.get(url=f'{root_url}{class_name}/{class_name}/brain_window/{fname}')
        img = Image.open(BytesIO(response.content))
        img = ImageOps.grayscale(img)
        img = np.asarray(img)
        img = np.reshape(img, (512*512))
        img_list.append(img)

    img_df = pd.DataFrame({'image_matrix': img_list})
    clean_intra = pd.concat([df, img_df], axis=1)
    img_list = [] #reset the image list for the next dataframe

In [19]:
#adding images to the cleaned multiple dataframe
root_url = 'https://raw.githubusercontent.com/Red-Nova/MATH4570-Project-Data/main/BrainCT/'
for class_name,df in ({'multi': clean_multiple}).items():

    for idx,fname in enumerate(df['Origin']):
        #for now, only process the brain window
        response = requests.get(url=f'{root_url}{class_name}/{class_name}/brain_window/{fname}')
        img = Image.open(BytesIO(response.content))
        img = ImageOps.grayscale(img)
        img = np.asarray(img)
        img = np.reshape(img, (512*512))
        img_list.append(img)

    img_df = pd.DataFrame({'image_matrix': img_list})
    clean_multiple = pd.concat([df, img_df], axis=1)
    img_list = [] #reset the image list for the next dataframe

In [20]:
#adding images to the cleaned subarachnoid dataframe
root_url = 'https://raw.githubusercontent.com/Red-Nova/MATH4570-Project-Data/main/BrainCT/'
img_list = []
for class_name,df in ({'subarachnoid':clean_subarachnoid}).items():

    for idx,fname in enumerate(df['Origin']):
        #for now, only process the brain window
        response = requests.get(url=f'{root_url}{class_name}/{class_name}/brain_window/{fname}')
        try:
            img = Image.open(BytesIO(response.content))
            img = ImageOps.grayscale(img)
            img = np.asarray(img)
            img = np.reshape(img, (512*512))
            img_list.append(img)
        except (UnidentifiedImageError):
            continue

    img_df = pd.DataFrame({'image_matrix': img_list})
    clean_subarachnoid = pd.concat([df, img_df], axis=1)
    img_list = [] #reset the image list for the next dataframe

In [21]:
#adding images to the cleaned subarachnoid dataframe
root_url = 'https://raw.githubusercontent.com/Red-Nova/MATH4570-Project-Data/main/BrainCT/'
class_to_df_dict = {'epidural':clean_epi,'intraparenchymal':clean_intra, 'multi': clean_multiple, 'subarachnoid':clean_subarachnoid,'subdural':clean_subdural2}
img_list = []
for class_name,df in ({'subdural':clean_subdural2}).items():

    for idx,fname in enumerate(df['Origin']):
        #for now, only process the brain window
        response = requests.get(url=f'{root_url}{class_name}/{class_name}/brain_window/{fname}')
        try:
            img = Image.open(BytesIO(response.content))
            img = ImageOps.grayscale(img)
            img = np.asarray(img)
            img = np.reshape(img, (512*512))
            img_list.append(img)
        except (UnidentifiedImageError, ValueError):
            continue

    img_df = pd.DataFrame({'image_matrix': img_list})
    clean_subdural2 = pd.concat([df, img_df], axis=1)
    img_list = [] #reset the image list for the next dataframe

In [22]:
clean_subarachnoid.head()

,Case ID,Origin,URL,Labeling State,All Labels,Total Reads,Total Qualified Reads,Majority Label,Number of ROIs,Correct Label,Difficulty,Agreement,Comments,Internal Notes,label,image_matrix
0,1197112.0,ID_001b1b944.jpg,https://go.centaurlabs.com/problem/1197112,In Progress,"['[{""x"": 0.43056944, ""y"": 0.5936399}, {""x"": 0....",16.0,5.0,"[{""x"": 0.314453125, ""y"": 0.75}, {""x"": 0.314453...",3.0,NaN,NaN,0.252748,NaN,NaN,3.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1197112.0,ID_001b1b944.jpg,https://go.centaurlabs.com/problem/1197112,In Progress,"['[{""x"": 0.43056944, ""y"": 0.5936399}, {""x"": 0....",16.0,5.0,"[{""x"": 0.400390625, ""y"": 0.796875}, {""x"": 0.40...",3.0,NaN,NaN,0.252748,NaN,NaN,3.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1197112.0,ID_001b1b944.jpg,https://go.centaurlabs.com/problem/1197112,In Progress,"['[{""x"": 0.43056944, ""y"": 0.5936399}, {""x"": 0....",16.0,5.0,"[{""x"": 0.556640625, ""y"": 0.302734375}, {""x"": 0...",3.0,NaN,NaN,0.252748,NaN,NaN,3.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1197367.0,ID_005f428d2.jpg,https://go.centaurlabs.com/problem/1197367,Labeled,"['[{""x"": 0.59604716, ""y"": 0.4975766}, {""x"": 0....",29.0,5.0,"[{""x"": 0.54296875, ""y"": 0.763671875}, {""x"": 0....",2.0,"[[{'x': 0.626953125, 'y': 0.30078125}, {'x': 0...",0.33,0.763632,NaN,NaN,3.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1197367.0,ID_005f428d2.jpg,https://go.centaurlabs.com/problem/1197367,Labeled,"['[{""x"": 0.59604716, ""y"": 0.4975766}, {""x"": 0....",29.0,5.0,"[{""x"": 0.630859375, ""y"": 0.30078125}, {""x"": 0....",2.0,"[[{'x': 0.626953125, 'y': 0.30078125}, {'x': 0...",0.33,0.763632,NaN,NaN,3.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [30]:
# combining all the dataframes
combined = clean_epi.append(clean_intra)
combined = combined.append(clean_multiple)
combined = combined.append(clean_subarachnoid)
combined = combined.append(clean_subdural2)
combined

C:\Users\sahsa\AppData\Local\Temp\ipykernel_31712\3476849609.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined = clean_epi.append(clean_intra)
C:\Users\sahsa\AppData\Local\Temp\ipykernel_31712\3476849609.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined = combined.append(clean_multiple)
C:\Users\sahsa\AppData\Local\Temp\ipykernel_31712\3476849609.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined = combined.append(clean_subarachnoid)
C:\Users\sahsa\AppData\Local\Temp\ipykernel_31712\3476849609.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined = combined.append(clean_subdural2)


,Case ID,Origin,URL,Labeling State,All Labels,Total Reads,Total Qualified Reads,Majority Label,Number of ROIs,Correct Label,Difficulty,Agreement,Comments,Internal Notes,label,image_matrix
0,1196514.0,ID_004c4b319.jpg,https://go.centaurlabs.com/problem/1196514,Labeled,"['[{""x"": 0.40818718, ""y"": 0.8623607}, {""x"": 0....",19.0,5.0,"[{""x"": 0.537109375, ""y"": 0.89453125}, {""x"": 0....",1.0,"[[{'x': 0.537109375, 'y': 0.89453125}, {'x': 0...",0.00,0.968851,NaN,NaN,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1196682.0,ID_0237b835f.jpg,https://go.centaurlabs.com/problem/1196682,Ready,"['[{""x"": 0.62959146, ""y"": 0.720186}, {""x"": 0.6...",51.0,28.0,"[{""x"": 0.654296875, ""y"": 0.724609375}, {""x"": 0...",1.0,NaN,NaN,0.063519,NaN,NaN,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1196477.0,ID_02e2d273d.jpg,https://go.centaurlabs.com/problem/1196477,Labeled,"['[{""x"": 0.35205495, ""y"": 0.34680602}, {""x"": 0...",22.0,6.0,"[{""x"": 0.46484375, ""y"": 0.357421875}, {""x"": 0....",1.0,"[[{'x': 0.46484375, 'y': 0.357421875}, {'x': 0...",0.06,0.710300,NaN,NaN,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1196612.0,ID_02f37ce84.jpg,https://go.centaurlabs.com/problem/1196612,Ready,"['[{""x"": 0.3680166, ""y"": 0.48387372}, {""x"": 0....",51.0,29.0,"[{""x"": 0.431640625, ""y"": 0.607421875}, {""x"": 0...",2.0,NaN,NaN,0.264134,NaN,NaN,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1196612.0,ID_02f37ce84.jpg,https://go.centaurlabs.com/problem/1196612,Ready,"['[{""x"": 0.3680166, ""y"": 0.48387372}, {""x"": 0....",51.0,29.0,"[{""x"": 0.353515625, ""y"": 0.59765625}, {""x"": 0....",2.0,NaN,NaN,0.264134,NaN,NaN,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,1198414.0,ID_ff2f3dbb7.jpg,https://go.centaurlabs.com/problem/1198414,Labeled,"['[{""x"": 0.70854944, ""y"": 0.6837957}, {""x"": 0....",14.0,6.0,"[{""x"": 0.70703125, ""y"": 0.67578125}, {""x"": 0.7...",1.0,"[[{'x': 0.70703125, 'y': 0.67578125}, {'x': 0....",0.15,0.153092,NaN,NaN,5.0,NaN
741,1198334.0,ID_04ab5246e.jpg,https://go.centaurlabs.com/problem/1198334,Labeled,"['[{""x"": 0.56073856, ""y"": 0.6705539}, {""x"": 0....",13.0,6.0,[],0.0,"[[{'x': 0.560546875, 'y': 0.669921875}, {'x': ...",0.33,0.833333,NaN,NaN,5.0,NaN
743,1198467.0,ID_2ef211ac4.jpg,https://go.centaurlabs.com/problem/1198467,Labeled,"['[{""x"": 0.31171963, ""y"": 0.61459357}, {""x"": 0...",11.0,5.0,[],0.0,"[[{'x': 0.302734375, 'y': 0.62109375}, {'x': 0...",0.00,0.800000,NaN,NaN,5.0,NaN
227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [31]:
combined = combined[combined['label'].notna()]
combined = combined[combined['image_matrix'].notna()]

,Case ID,Origin,URL,Labeling State,All Labels,Total Reads,Total Qualified Reads,Majority Label,Number of ROIs,Correct Label,Difficulty,Agreement,Comments,Internal Notes,label,image_matrix
731,1198284.0,ID_fe39f38ec.jpg,https://go.centaurlabs.com/problem/1198284,Labeled,"['[{""x"": 0.5425447, ""y"": 0.5063234}, {""x"": 0.5...",14.0,5.0,"[{""x"": 0.568359375, ""y"": 0.7421875}, {""x"": 0.5...",3.0,"[[{'x': 0.568359375, 'y': 0.7421875}, {'x': 0....",0.35,0.364669,NaN,NaN,5.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
732,1198284.0,ID_fe39f38ec.jpg,https://go.centaurlabs.com/problem/1198284,Labeled,"['[{""x"": 0.5425447, ""y"": 0.5063234}, {""x"": 0.5...",14.0,5.0,"[{""x"": 0.544921875, ""y"": 0.75390625}, {""x"": 0....",3.0,"[[{'x': 0.568359375, 'y': 0.7421875}, {'x': 0....",0.35,0.364669,NaN,NaN,5.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
733,1198284.0,ID_fe39f38ec.jpg,https://go.centaurlabs.com/problem/1198284,Labeled,"['[{""x"": 0.5425447, ""y"": 0.5063234}, {""x"": 0.5...",14.0,5.0,"[{""x"": 0.580078125, ""y"": 0.62109375}, {""x"": 0....",3.0,"[[{'x': 0.568359375, 'y': 0.7421875}, {'x': 0....",0.35,0.364669,NaN,NaN,5.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
734,1198089.0,ID_fe75573b1.jpg,https://go.centaurlabs.com/problem/1198089,In Progress,"['[{""x"": 0.27365866, ""y"": 0.5667272}, {""x"": 0....",14.0,6.0,"[{""x"": 0.2734375, ""y"": 0.564453125}, {""x"": 0.2...",1.0,NaN,NaN,0.315243,NaN,NaN,5.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
735,1198118.0,ID_fec1d488a.jpg,https://go.centaurlabs.com/problem/1198118,In Progress,"['[{""x"": 0.27544808, ""y"": 0.37043557}, {""x"": 0...",15.0,5.0,"[{""x"": 0.271484375, ""y"": 0.375}, {""x"": 0.27148...",1.0,NaN,NaN,0.648669,NaN,NaN,5.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [70]:
X = np.array(combined['image_matrix'])
y = np.array(combined['label'])

data = np.zeros([4633, 262144])
for n, image in enumerate(X):
    data[n,:] = image

In [72]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=0)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.25, random_state=0)

print(y_train.shape, y_test.shape, y_valid.shape)

MemoryError: Unable to allocate 7.24 GiB for an array with shape (3706, 262144) and data type float64

In [ ]:
DS = 16

if 262144/DS % 1 > 0:
    print("Downsample rate is not a multiple of 262144")
    DS = 1
    im_size = 262144
else:
    im_size = int(262144/DS)

N_train = y_train.shape[0]

Xds_train = np.zeros([N_train * DS, im_size])
yds_train = np.zeros([N_train * DS, 1])

for n in range(N_train):
    for i in range(DS):
        Xds_train[n+1,:] = X_train[n,i::DS]
        yds_train[n+1] = y[n]

N_test = y_test.shape[0]

Xds_test = np.zeros([N_test * DS, im_size])
yds_test = np.zeros([N_test * DS, 1])

for n in range(N_test):
    for i in range(DS):
        Xds_test[n+1,:] = X_test[n,i::DS]
        yds_test[n+1] = y[n]

N_valid = y_valid.shape[0]

Xds_valid = np.zeros([N_valid * DS, im_size])
yds_valid = np.zeros([N_valid * DS, 1])

for n in range(N_valid):
    for i in range(DS):
        Xds_valid[n+1,:] = X_valid[n,i::DS]
        yds_valid[n+1] = y[n]

print(Xds_train.shape, yds_train.shape)
print(Xds_test.shape, yds_test.shape)
print(Xds_valid.shape, yds_valid.shape)

In [69]:
X_train = Xds_train.reshape(-1, 32, 32, 1)
X_test = Xds_test.reshape(-1, 32, 32, 1)
X_valid = Xds_valid.reshape(-1, 32, 32, 1)

ValueError: cannot reshape array of size 971505664 into shape (44,44,1)

In [65]:
import tensorflow as tf
from tensorflow import keras

np.random.seed(44)

K = keras.backend

class ExponentialLearningRate(keras.callbacks.Callback):
    def init(self, factor):
        self.factor = factor
        self.rates = []
        self.losses = []
    def on_batch_end(self, batch, logs):
        self.rates.append(K.get_value(self.model.optimizer.lr))
        self.losses.append(logs["loss"])
        K.set_value(self.model.optimizer.lr, self.model.optimizer.lr * self.factor)

In [66]:
model_cnn = keras.models.Sequential()
model_cnn.add(keras.layers.Conv2D(32, (3,3), padding = 'valid', input_shape=(32,32,1), activation = 'relu'))
model_cnn.add(keras.layers.MaxPooling2D((2,2)))
model_cnn.add(keras.layers.Conv2D(64, (3,3), padding = 'valid', activation = 'relu'))
model_cnn.add(keras.layers.MaxPooling2D((2,2)))
model_cnn.add(keras.layers.Flatten())
model_cnn.add(keras.layers.Dense(32))
model_cnn.add(keras.layers.Dropout(0.5))
model_cnn.add(keras.layers.Dense(6))

model_cnn.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], 
                  optimizer=keras.optimizers.SGD(learning_rate=1e-3))
expon_lr_cnn = ExponentialLearningRate()
expon_lr_cnn.factor = 1.001
model_cnn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 30, 30, 32)        320       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 6, 6, 64)         0         
 g2D)                                                            
                                                                 
 flatten_8 (Flatten)         (None, 2304)              0         
                                                                 
 dense_16 (Dense)            (None, 32)               

In [67]:
print(X_train.shape)
print(yds_train.shape)

(948736, 32, 32, 1)
(59296, 1)


In [63]:
history_cnn = model_cnn.fit(X_train, yds_train, batch_size=64, epochs=1, verbose=1, validation_data=(X_valid, yds_valid), 
                          callbacks=[expon_lr_cnn])

ValueError: Data cardinality is ambiguous:
  x sizes: 948736
  y sizes: 59296
Make sure all arrays contain the same number of samples.